# How much should properties be worth in Milwaukee, Wisconsin? (Part II)

## Goals

In the last few cases, we have seen several techniques used to diagnose the applicability of linear regression models. We have learned how to use indicators such as AIC to compare and detect overcomplicated models, how a proper study of the residuals can help to fit our models better, and how to extend the applicability of linear models with the use of variable transformations. The goal of this case is to learn a couple of tools that will help us further understand the applicability of our models better:

1. The use of test/training sets to evaluate the performance of our models
2. Detection of **multicolinearity** to improve numerical stability and increase the robustness of the predictive model

## Introduction

**Business Context.** You are the same property developer from the previous case, with the same goal. Although the previous model you built was a good start, it did not incorporate all the variables you wished to include, and you are skeptical of how well it might work on data that it was not trained on; i.e. **out-of-sample data**.

**Business Problem.** Your task is to **improve your model to predict property prices in the city of Milwaukee, Wisconsin**.

**Analytical Context.** Again, the dataset consists of property sales (commercial and residential) in Milwaukee, Wisconsin from 2002 to 2018. focused on diagnosing and fixing possible issues arising in linear regression. In the previous case, We learned how to visually analyze residuals and to detect outliers and heteroscedasticity. We showed that variable transformation can improve some of these issues, while explicit removal of outliers *explainable by external factors* could improve things further.

This case is structured as follows: you will 1) learn how categorical variables are actually handled in regression models; 2) dive into the predictive ability of the model and learn how to assess and improve it; and finally 3) look at **multicollinearity**, an issue that arises when fitting regression models with highly correlated or too many predictors and how to deal with it.

## Preparing our data

We will pick up were we left things off at the end of the last case. We will load the same packages and data, and fit the same model. The one difference is that we will remove from the dataset all properties with a sale price
below $2,000, which may not correspond to real market prices, per our analysis of residuals in the last exercise of the previous case:

In [ ]:
### Load relevant packages
import pandas                  as pd
import numpy                   as np
import matplotlib.pyplot       as plt
import seaborn                 as sns
import statsmodels.formula.api as smf
import statsmodels.api         as sm
import scipy

%matplotlib inline
plt.style.use('ggplot')
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [ ]:
data = pd.read_csv("2002-2018-property-sales-data.csv",
    dtype = { # indicate categorical variables
        "PropType": "category",
        "District": "category",
        "Extwall": "category",
        "Nbhd": "category",
        "Style": "category",
    },
    parse_dates=["Sale_date"], # the Sale_date column is parsed as a date
)
def remove_unused_categories(data):
    """ The `remove_unused_categories` method in pandas
        removes categories from a Series if there are no
        elements of that category.
        
        This function is a convenience function that removes
        unused categories for all categorical columns
        of a data frame.
        
        The reason this is useful is that when we
        fit a linear regression, `statsmodels` will
        create a coefficient for every category in a column,
        and so unused categories pollute the results.
    """
    for cname in data:
        col = data[cname]
        if pd.api.types.is_categorical_dtype(col):
            data[cname] = col.cat.remove_unused_categories()
    return data

clean = np.where(
    (data["Sale_price"] > 2000) & # this is the only change!
    (data["Year_Built"] > 1800) &
    (data["Fin_sqft"] > 0) & # must have non-zero finished square feet
    (data["Lotsize"] > 0)  & # must have non-zero lot size
    (data["PropType"] == "Residential")
    )
data_clean = data.iloc[clean].copy()
remove_unused_categories(data_clean).head()

## Training vs. test sets

As we discussed in the introduction, one of our chief concerns is whether or not the model we have built works just as well on out-of-sample data as it does on in-sample data. This is a *very* common problem in model-building, as is known as **overfitting**. You will learn more about overfitting in future cases, but here we will discuss a simple method for mitigating it.

The idea is to randomly split the data into a training set and a test set. The **training set** is the one on which we train and fit our multiple linear regression model. We then run our fitted model on the **test set** and compared its predictions against the actual test set response variable data to evaluate its performance.

### Exercise 1:

Write code to split the data into training and test sets (an 80-20 split is a good starting point). Fit a linear regression model on the training set, with the logarithm of `Sale_price` as the response variable and `District`, `Units`, and the logarithm of `Fin_sqft` as the predictor variables.'

**Note:** In order to make the results reproducible, write ```np.random.seed(135568109)``` before your code.

**Answer.**

-------

### Categorical features under the hood

You may have noticed that there are over a dozen coefficients for `District` above. This is because `District` is a categorical variable, and for categorical features, one coefficient is obtained for all but one of the categories. If there are only two categories (e.g. gender), this is intuitive: the feature is converted into a column of zeros and ones before feeding it into the regression, where it is treated like a regular numerical variable.

When there are more than two categories, one category is designated the “reference” or “baseline” category, and “dummy” columns of ones and zeros are created for all other categories. Let's take a dummy example with three categories and five rows:

| Category        |
|--------------|
| A            |
| B            |
| C            |
| C            |
| A            |

Before the linear regression is fitted, the `Category` column is transformed into **two** “dummy” columns (a column for baseline category is not added).

The first column is 1 if the district is `B`, and 0 otherwise, whereas the second column is 1 if the district is `C`, and 0 otherwise.  We get:

| Category_B            | Category_C           |
|--------------|-------------|
| 0            | 0           |
| 1            | 0           |
| 0            | 1           |
| 0            | 1           |
| 0            | 0           |

The dummy columns are fed into the linear regression and treated like regular numerical variables. This technique is called **one-hot encoding** and can be done manually with the pandas function `pd.get_dummies()`:

In [ ]:
df_dummy=pd.DataFrame({'Category':['A','B','C','C','A']})
pd.get_dummies(df_dummy,columns=['Category'], drop_first=True) #Use drop_first to drop the first category

Just like in the binary case, the choice of baseline changes the coefficients and their interpretation; the `District[T.3]` coefficient of 1.0236 should be interpreted as the difference in predicted outcomes between districts 3 and 1. But the baseline choice does not affect predictions and model performance, which is why most software will arbitrarily pick one category as the baseline without warning (often the first alphabetically).

### Exercise 2:

#### 2.1

Add `Style` to our last regression model. Do this in two ways: first by transforming the `Style` variable using `pd.get_dummies()` and then without the transformation. Verify that you obtain the same results. According to this model, which property style is the most desirable?

**Hint:** The `Style` variable contains names, like `Residence O/S`, which may be problematic when writing `smf.ols` formulas (see the [Pasty Syntax](https://patsy.readthedocs.io/en/latest/index.html)). You can use the `pasty` quote `Q()` syntax on variable names to avoid this issue:
```
formula="np.log(Sale_price) ~Q("Residence O/S")+...
```

**Answer.**

-------

#### 2.2

In the model below, we have *not* removed the baseline category after using ```get_dummies()```. Is this model correct? Why or why not? 

In [ ]:
train_dummy2=pd.get_dummies(train,columns=['Style'],prefix='',prefix_sep='')
formula1="np.log(Sale_price) ~ "
for style in Styles:
    formula1+='Q("'+style+'")+'
formula1=formula1+'District + Units+ np.log(Fin_sqft)'

#Create the model
model_style_2 = smf.ols(formula = formula1, 
                 data = train_dummy2).fit()
model_style_2.summary()

**Answer.**

-------

## Evaluating predictive performance

Recall that for any given data point, the residual is the difference between our model's prediction for that point and the actual value of that point. The **mean absolute error (MAE)** is a metric which summarizes the model's holistic performance on the entire dataset. The MAE is calculated by taking the absolute value of each residual, then taking the mean of all of those absolute values. In essence, the MAE describes the typical magnitude of the residuals; the lower the MAE, the better.

In [ ]:
def MAE(prediction,true_values):
    return np.mean(                                                      # Mean
                np.abs(                                                   # Absolute
                        prediction-true_values                            # Error
                    )
                )

print("MAE between model_log and log of true Sale price:",MAE(model_log.predict(test) ,np.log(test.Sale_price)))
print("MAE between exp(model_log) and true Sale price:",MAE(np.exp(model_log.predict(test)) ,test.Sale_price))

Another very popular choice for evaluating regression is the **root mean squared error (RMSE)**. This is computed by taking the square of each residual, averaging them, and then taking the square root. Below is the implementation in python, which is possibly easier to understand than the above explanation:

In [ ]:
def RMSE(prediction,true_values):
    
    return np.sqrt(                                                          # Root
            np.mean(                                                      # Mean
                np.square(                                                # Squared
                         prediction-true_values                           # Error
                )
            )
        )
        
print("RMSE between model_log and log of true Sale price:", RMSE(model_log.predict(test) ,np.log(test.Sale_price)))
print("RMSE between exp(model_log) and true Sale price:",RMSE(np.exp(model_log.predict(test)) ,test.Sale_price))

Finally, there is the **mean absolute percentage error**. This takes the absolute value of each residual and divides it by the actual value of that point to obtain a percentage, then averaging across all percentages.

In [ ]:
def MAPE(prediction,true_value):
    return np.mean(                                           # Mean
        np.abs(                                               # Absolute
               (prediction-true_value)/true_value             # Error
            )*100                                            # Percentage
    )

print("MAPE between model_log and log of true Sale price:", MAPE(model_log.predict(test) ,np.log(test.Sale_price)))
print("MAPE between exp(model_log) and true Sale price:", MAPE(np.exp(model_log.predict(test)) ,test.Sale_price))

### Question:

Why would we not just use MAE instead of having to deal with three different metrics?

Although RMSE may seem needlessly complicated compared to MAE, it is much more commonly used. The reason is that the RMSE metric is the same as the one being minimized on the training data by a standard linear regression model (also called an **ordinary least squares (OLS)** regression). This makes it a “natural” choice using the same metric to evaluate the out-of-sample (test) performance.

Furthermore, the RMSE puts much more weight on outliers, since the errors are squared before being averaged. In cases where outliers are especially bad and need to be punished, the RMSE is a better choice.

The MAPE has a nice interpretation in that we can say that a model's predictions are on average wrong by a certain percentage. For example, our house price model could be on average off by 30%. Such a model is often described as being "70% accurate".

For those who are mathematically inclined, the equations below define the three error metrics above:

\begin{eqnarray*}
    MAE &= \frac{1}{n} \sum_{i=1}^n \left| y_i - \hat{y}_i \right| \\
    RMSE &= \sqrt{ \frac{1}{n} \sum_{i=1}^n ( y_i - \hat{y}_i )^2 } \\
    MAPE &= 100\% \frac{1}{n} \sum_{i=1}^n \left|\frac{ y_i - \hat{y}_i}{y_i} \right|
\end{eqnarray*}

### Exercise 3:

#### 3.1 

Compute the MAE, RMSE and MAPE for the linear model 
``` 
"Sale_price ~ District + Units+ Fin_sqft"
```
How does this model compares to ```model_log```?

**Answer.**

-------

#### 3.2

Let us illustrate the differences between MAE, RMSE and MAPE with an example. In the following data, our test sample has been contaminated with a single outlier. Compare the MAE, RMSE and MAPE of the models:
```
model1: y~x
model2: y~(7.8182)x-24.3557
```
Use the test data ```x_test, y_test``` provided below. What do you see? Which model is better?

In [ ]:
np.random.seed(135568109)
error=np.random.normal(0,0.1,100)
x_data=np.arange(0,10,0.1)
y_data=x_data+error

#Split the data into training and test data:
train_ex=np.random.randint(0,99,80)
x_train=x_data[train_ex]
y_train=y_data[train_ex]

test_ex=list(set(range(100))-set(train_ex))
x_test=x_data[test_ex]
y_test=y_data[test_ex]

#oops it got contaminated
x_test=np.append(x_test,11)
y_test=np.append(y_test,1000)


fig, axes=plt.subplots(nrows=1, ncols=2,figsize=(16,8))
sns.regplot(x=x_train, y=y_train,ax=axes[0])
axes[0].set_title('Training Data')

sns.regplot(x=x_test, y=y_test,ax=axes[1])
axes[1].set_title('Test Data')

**Answer.**

-------

## Adding additional predictors to improve our model

Consider now `model_feat` described below. This model adds features for the number of stories and bedrooms, the year built, neighborhood (instead of simply district), type of external wall, number of full and half baths, lot size and sale year. This last predictor is an example of **feature engineering** – remember we are not limited to the predictors as they are given to us in the raw data. Often, it is beneficial to transform and combine predictors, as we did here by extracting the year from the sale date and treating it as a categorical variable.

### Exercise 4:

#### 4.1

Fit a linear regression of the logarithm of the sale price with the following features:

1. number of stories
2. number of bedrooms
3. number of units in the property
4. neighborhood code
5. style
6. type of external wall
7. number of full baths
8. number of half baths
9. the logarithm of square footage
10. the logarithm of the lot size
11. year the property was *built* as a *numerical* variable
12. year the property was *sold* as a *categorical* variable

Give the resulting fitted model the variable name `model_feat`.

**Hint:**

1) write a function to extract the year from the sale date – `train.Sale_date.iloc[0].year` gives the year of the first sale; 

2) [use `C()` around a term in the formula to make the term categorical](https://patsy.readthedocs.io/en/latest/categorical-coding.html))

**Answer.**

-------

#### 4.2

How should the coefficient `Extwall[T.Brick]` of 0.0236 be interpreted? Select the appropriate choice below:

"A property with brick external walls is predicted to have a log price that is..."

(a) 0.0236 higher than a property with aluminum / vinyl external walls that is otherwise identical

(b) increased by 0.0236, all else being equal

(c) 0.0236 higher than the average property in the training dataset

**Answer.**

-------

#### 4.3

Compute the mean absolute error for `model_feat`, and compare it to `model_log`. Does `model_feat` perform better or worse? What about the $R^2$ and AIC of these two models?

**Answer.**

-------

#### 4.4
What story does this model tell us about the Milwaukee housing market during the last 15 years?

(a) The prices increased until 2007, when there was a housing crash. Since then, prices have gone back up to their pre-crash level

(b) Overall, housing prices steadily increased over time

(c) The prices increased until 2007, then subsequently went down

(d) The prices increased until 2007, when there was a housing crash. Since then, prices have slowly risen, but they have not yet fully recovered

**Answer.**

-------

#### 4.5

A 2-story townhouse is on the market for $150,000 with the following characteristics:

1. Neighborhood 4320 in District 11
2. 1200 finished square feet
3. 7200 square feet lot size
4. 3 bedrooms
5. Vinyl external wall
6. built in 1987
7. 1 full bath and 1 half-bath
8. 1 unit

Is this a good price according to our model? What assumptions do you have to make to answer this question?

**Answer.**

-------

## Collinearity and standard errors

The model in the previous section seems to be a good step towards our main goal. However, from the last few cases we have learned that adding a large dump of new features to a model and then leaving it to its own devices is not a prudent strategy, because:

1. We do not want to include features that add little predictive power to our model. Otherwise, we may end up with an overcomplicated model. Remember that we generally want a model that achieves a good level of predictiveness without sacrificing simplicity
2. Adding too many features increases the danger of *overfitting*, thus decreasing the potential applicability of the model

So far, we have learned a couple of methods to alleviate these issues:

1. We can use indicators such as the ```AIC``` to detect if we are adding variables with little predictive power
2. We can divide our sample data into `Train/Test` sets and use the `Test` set, along with some measure like `RMSE`, to evaluate the applicability of our model

However, there exists a third potential issue whenever we add too many features to our models: 

3. Linear regression can actually be **destabilized** when there are too many predictors that do not add value

This means that small changes in the training data fed into the model results in large changes in the fitted coefficients; overfitting is a common consequence of this phenomenon.

### Question:

What do you think causes the above to happen?

As one adds more predictors, they begin to contribute redundant information (which explains why every additional predictor contributes zero marginal information). Mathematically, this means that new predictors can often be expressed to a good extent as linear combinations (weighted sums) of some of the other predictors. When this happens, a model is said to exhibit **collinearity** or **multicollinearity**.

Since predictors become mostly redundant, this means that nearly identical predictions can be obtained by vastly different coefficient values, by "trading off" between the coefficients of the predictors which exhibit collinearity.
Consequently, the model has no way to decide from the data alone which of these possible models with identical predictions to choose, which translates into inflated standard errors and a high risk of overfitting.

To demonstrate this effect, we will introduce an engineered feature: `log(Fin_sqft/Lotsize + 0.00001)`.
The small constant is there to avoid making the new feature *perfectly* collinear with log(`Fin_sqft`) and log(`Lotsize`) (which would create a "divide-by-zero" error in the background regression function code):

In [ ]:
# base model without engineered collinear feature
model_finlot = smf.ols(formula = "np.log(Sale_price) ~ "
                           "Stories + Year_Built"
                           "+ np.log(Fin_sqft)"
                           "+ np.log(Lotsize)"
                           "+ Style"
                           "+ Extwall  + Units + Bdrms"
                           "+ Fbath + Hbath + Nbhd"
                           "+ C(year_from_date(Sale_date))", 
                 data = train).fit()
model_finlot.summary()

In [ ]:
model_coll = smf.ols(formula = "np.log(Sale_price) ~ "
                           "Stories + Year_Built"
                           "+ np.log(Fin_sqft)"
                           "+ np.log(Lotsize)"
                           "+ np.log(Fin_sqft/Lotsize + 0.0001)"
                           "+ Style"
                           "+ Extwall  + Units + Bdrms"
                           "+ Fbath + Hbath"
                           "+ Nbhd"
                           "+ C(year_from_date(Sale_date))", 
                 data = train).fit()
model_coll.summary()

### Exercise 5:

#### 5.1

What happened to the coefficients and standard errors of `log(Fin_sqft)` and `log(Lotsize)` as the engineered feature was introduced?

**Answer.**

-------

#### 5.2: 

What happens when you change the 0.0001 constant (say, to 0.00001 or 0.001) in the engineered feature?

**Answer.**

-------

To detect and investigate collinearities, it is useful to look at correlations between predictors within a model, shown below. Notice the high correlations (positive *or* negative) between the engineered feature `np.log(Fin_sqft / Lotsize + 0.0001)` and `np.log(Fin_sqft)`/`np.log(Lotsize)`. Note also the other high correlations with bathrooms and number of stories in the matrix. These can destabilize the model and make it difficult to make meaningful statements about the incremental value of adding extra bathrooms and stories.

In [ ]:
M = model_coll.model.data.exog # the design matrix X containing all
                               # predictors including dummy variables
                               # for categorical variables
param_names = model_coll.model.data.param_names # parameter names
M_df = pd.DataFrame(M, columns=param_names) # as a pandas data frame
# subset of predictors to plot:
M_sub = M_df[['np.log(Fin_sqft)','np.log(Lotsize)',
              'np.log(Fin_sqft / Lotsize + 0.0001)',
              'Bdrms','Fbath','Hbath','Stories']]
plt.figure(figsize=(6,5))
sns.heatmap(M_sub.corr(), cmap="RdYlBu", 
    annot=True, square=True,
    vmin=-0.8, vmax=0.8, fmt="+.1f")
plt.title("Correlations between predictors")

This illustrates the danger of just throwing additional features into a model. Not only is it redundant, but it can also lead to highly unstable and nonsensical values for the coefficients, which can lead to overfitting in production systems. One way of dealing with this is what we discussed in previous cases – using the AIC criterion to iteratively evaluate each new feature and determine if it is really adding enough incremental value on top of a penalty for the additional complexity. Another is to examine correlation matrices like the one we created above to identify any values close to -1 or 1. In future cases on **cross-validation** and **regularization**, you will learn additional ways to deal with overly complex and potentially overfit models.

## Conclusions

In this case, we had a closer look at categorical predictors. We then focused on predictions, and discussed a few different metrics for the predictive power of a regression model. Each had their own advantages and disadvantages depending on what the business problem at hand needed to prioritize.

To improve the predictive power of our model, we added additional predictors. We learned how to **engineer features** to capture information that was not directly available as a column of the data set. However, we saw how adding so many features at once could lead to **collinearity** and therefore destabilize a model, making coefficients difficult to interpret and increasing the chance of overfit predictions in production.

## Takeaways

In this case, we have expanded our toolbox for the diagnosis of linear regression models. We have learned:
    
1. How the use of training/testing sets, along with measures such as MAE, MAPE and RMSE can help in evaluating the predictive power of linear models. We note that these tools are not only applicable to linear models, but also work with any other type of model and we will continue using them in future cases. 
2. How categorical variables are encoded in linear regression models.
3. That more complex models can provide better predictions, but this can destabilize performance. In many such situations, complexity creates multicollinearity between variables which actually makes our models highly unstable and prone to overfitting. 

Remember, much of the business world is about applying trained models on future, currently unknown data. Overfitting can lead to serious negative impacts on the bottom line.

## Attribution

"Property sales data, 2002-2018 Master File", April 29, 2019, Milwaukee OpenData, Creative Commons Attribution, https://data.milwaukee.gov/dataset/property-sales-data/resource/f083631f-e34e-4ad6-aba1-d6d7dd265170
